In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

# LLM Chain

In [4]:
llm.invoke("Did Greece land troops in Anatolia region in Turkey?")

"A great question!\n\nIn ancient times, yes, the Greeks did have military presence and colonies in parts of western Asia Minor (Anatolia), which is now part of modern-day Turkey. Here are some examples:\n\n1. **Ionian cities**: In the 8th century BCE, Greek colonizers established city-states like Ephesus, Colophon, and Smyrna (now İzmir) on the Aegean coast of Anatolia.\n2. **Aeolic colonies**: Around the same time, other Greek colonizers founded cities like Lesbos, Adramyttium, and Abydos in northwestern Anatolia.\n3. **Milesian colonization**: The city-state of Miletus (in modern-day Turkey) sent out colonists to establish settlements in western Anatolia, including Ephesus and Colophon.\n\nThese Greek colonies in Anatolia were an important part of the Greek world, as they facilitated trade and cultural exchange with other Mediterranean civilizations. However, it's essential to note that these colonies were not necessarily established by armies or by force; rather, they were often fou

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm excited to share some ways Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith's AI-powered writing capabilities can generate test cases, test scenarios, and even test scripts! This helps ensure your software or system meets the desired functionality and performance standards.\n\n2. **Test Case Documentation**: When you use Langsmith to generate test cases, you'll also get high-quality documentation that explains each test scenario in detail. This makes it easy for your team to understand the testing process and ensures that everyone is on the same page.\n\n3. **Error Message Generation**: In the event of a failure or error, Langsmith can help generate clear and concise error messages that guide users towards a solution. This reduces user frustration and improves overall user experience.\n\n4. **Test Data Generation**: Langsmith's AI can also assist in generating test data, such as input values, configuration set

In [8]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [11]:
chain.invoke({"input": "how can langsmith help with testing?"})

"What a great question!\n\nAs a world-class technical documentation writer, I'm delighted to share some insights on how Langsmith can assist with testing.\n\nLangsmith is an AI-powered writing tool that specializes in generating high-quality text based on input prompts. Here are some ways it can support your testing efforts:\n\n1. **Automated Testing Scripts**: Langsmith can generate test scripts in various programming languages (e.g., Python, Java) by analyzing the requirements and design documents. This helps reduce the manual effort required to create test cases.\n2. **Test Data Generation**: Langsmith's AI capabilities enable it to generate relevant test data, such as input values, expected outputs, or error messages. This saves time and ensures that your test data is comprehensive and diverse.\n3. **Test Case Prioritization**: By analyzing the test cases generated by Langsmith, you can prioritize them based on risk, complexity, or business value. This helps focus your testing effo

# Retrieval Chain

In [12]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [17]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='llama3')

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
  {context}
</context>
Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [20]:
from langchain_core.documents import Document
document_chain.invoke({
   "input": "how can langsmith help with testing?",
   "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Based on the provided context, Langsmith can help with testing by allowing users to "visualize" their test results.'

In [21]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [22]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Rapid Prototyping**: LangSmith's playground environment allows for quick iteration and experimentation with different prompts and models.
2. **Debugging**: The platform provides native rendering of chat messages, functions, and retrieve documents, making it easier to identify and root-cause issues.
3. **Test Driven Approach**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
4. **Evaluation and Scoring**: The platform supports custom evaluations (both LLM and heuristic based) to score test results and track regressions across multiple revisions of the application.
5. **Comparison View**: LangSmith's comparison view allows users to see which variant is performing better by viewing results for different configurations on the same datapoints side-by-side.
6. **Automated Evaluations**: The pla

# Conversation Retrieval Chain